# Explore results after scraping results website

### User variables


In [1]:
Date = '2019-02' # yyyy-mm

### Modules and functions

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display,Image

### Load results

Load auction results and data from rdw.

In [3]:
file_name = '../data/drz-data-{}.pkl'.format(Date)
print(file_name)
drz = pd.read_pickle(file_name)
file_name = '../data/rdw-data-{}.pkl'.format(Date)
print(file_name)
rdw = pd.read_pickle(file_name)

../data/drz-data-2019-02.pkl
../data/rdw-data-2019-02.pkl


# Cheap cars with valid inspection

In [4]:
is_cheap = (
    drz.LotType == "Personenauto"
) & (
    drz.Price<3000
) & (
    drz.Price>0
) & ~(
    (
        drz.APKdate.isnull()
    ) | (
        drz.APKdate.apply(lambda x:x == '')
    )
)

drz.loc[is_cheap,
        ["Price","ItemType","ItemBrand","Mfdate","APKdate","OdoKM"]]

,Price,ItemType,ItemBrand,Mfdate,APKdate,OdoKM
2019-2-7107,1111.0,audi a2; 55 kw,AUDI,13.10.2000,08.06.2019,291.532
2019-2-7109,901.0,clk 200; coupe,MERCEDES-BENZ,15.08.1997,08.10.2019,193.127
2019-2-7118,1668.0,golf,VOLKSWAGEN,04.07.2008,20.07.2019,195.785
2019-2-7124,1689.0,polo; tdi 96 kw,VOLKSWAGEN,25.10.2005,17.04.2019,252.925
2019-2-7126,683.0,megane; gr tour 1.6 16v,RENAULT,07.10.2004,30.03.2019,249.575
2019-2-7128,1551.0,picanto; 1.0 m/t,KIA,18.06.2004,18.06.2019,130.630
2019-2-7133,1155.0,panda,FIAT,03.08.2007,01.04.2019,205.436
2019-2-7137,2226.0,tucson; 2.0 crdi aut,HYUNDAI,02.06.2006,01.06.2019,254.762
2019-2-7141,182.0,c5; 1.8i 16v,CITROËN,19.05.2001,20.07.2019,249.392
2019-2-7142,313.0,c5; 1.8i 16v,CITROËN,21.10.2004,29.05.2019,207.619


In [5]:
rdw.loc[is_cheap,["rdwinfo"]]

,rdwinfo
2019-2-7107,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7109,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7118,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7124,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7126,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7128,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7133,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7137,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7141,{'afstand_hart_koppeling_tot_achterzijde_voert...
2019-2-7142,{'afstand_hart_koppeling_tot_achterzijde_voert...


In [6]:
rdw.loc[is_cheap,"rdwinfo"][0]

{'TimeStamp': '20190208',
 'aantal_assen': {0: 2, 1: 2},
 'aantal_cilinders': {0: 4},
 'aantal_deuren': {0: 4},
 'aantal_rolstoelplaatsen': {0: 0},
 'aantal_wielen': {0: 4},
 'aantal_zitplaatsen': {0: 4},
 'afstand_hart_koppeling_tot_achterzijde_voertuig': {0: 0},
 'afstand_voorzijde_voertuig_tot_hart_koppeling': {0: 0},
 'api_gekentekende_voertuigen_assen': {0: 'https://opendata.rdw.nl/resource/3huj-srit.json'},
 'api_gekentekende_voertuigen_brandstof': {0: 'https://opendata.rdw.nl/resource/8ys7-d773.json'},
 'api_gekentekende_voertuigen_carrosserie': {0: 'https://opendata.rdw.nl/resource/vezc-m2t6.json'},
 'api_gekentekende_voertuigen_carrosserie_specifiek': {0: 'https://opendata.rdw.nl/resource/jhie-znh9.json'},
 'api_gekentekende_voertuigen_voertuigklasse': {0: 'https://opendata.rdw.nl/resource/kmfi-hrps.json'},
 'as_nummer': {0: 2, 1: 1},
 'brandstof_omschrijving': {0: 'Benzine'},
 'brandstof_volgnummer': {0: 1},
 'brandstofverbruik_buiten': {0: 4.8},
 'brandstofverbruik_gecombine

In [7]:
pd.DataFrame(rdw.loc[is_cheap,"rdwinfo"][0]).T

,0,1
TimeStamp,20190208,20190208
aantal_assen,2,2
aantal_cilinders,4,NaN
aantal_deuren,4,NaN
aantal_rolstoelplaatsen,0,NaN
aantal_wielen,4,NaN
aantal_zitplaatsen,4,NaN
afstand_hart_koppeling_tot_achterzijde_voertuig,0,NaN
afstand_voorzijde_voertuig_tot_hart_koppeling,0,NaN
api_gekentekende_voertuigen_assen,https://opendata.rdw.nl/resource/3huj-srit.json,NaN


# Pictures of lots

In [8]:
# ix=drz.N_images.idxmax()
ix=drz.Price.idxmax(); print('Price: EUR {:g}\n\n'.format(drz.loc[ix,'Price']))
# Mfdate = drz.Mfdate.apply(lambda x: pd.to_datetime(x,format='%d.%m.%Y') if len(x)==10 else pd.np.NaN)
# ix=Mfdate.idxmin(); print('year of mf: {:s}\n\n'.format(drz.loc[ix,'Mfdate']))

print(ix)
for u in drz.loc[ix,'Images']:
    print(u)
    print(drz.loc[ix,'Source'])
    display(Image(url=u,width=400))
    

Price: EUR 35290


2019-2-2405
http://www.domeinenrz.nl/ufc/static/1548759409/thumb/domeinenrz_sites/f34d92684bb2c7d3869a7934a3e0f5ea/1024/768/image.jpg
http://www.domeinenrz.nl/catalogus/verkoop_bij_inschrijving_2019-0002_februari?=&meerfotos=K1900022405&veilingen=2019-0002


http://www.domeinenrz.nl/ufc/static/1548759407/thumb/domeinenrz_sites/d61eb3ff6bdb8bb975e302e1ce27292b/1024/768/image.jpg
http://www.domeinenrz.nl/catalogus/verkoop_bij_inschrijving_2019-0002_februari?=&meerfotos=K1900022405&veilingen=2019-0002


http://www.domeinenrz.nl/ufc/static/1548759407/thumb/domeinenrz_sites/d70df81850af1aa7192cd4455be593e7/1024/768/image.jpg
http://www.domeinenrz.nl/catalogus/verkoop_bij_inschrijving_2019-0002_februari?=&meerfotos=K1900022405&veilingen=2019-0002
